In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from uproot3_methods.classes.TLorentzVector import TLorentzVector

In [ ]:
file = up.open("../build/evtOutput.root")

In [ ]:
class rootDF:
    def __init__(self, file_name):
        file = up.open(file_name)
        ntp = file["ntp"]
        self.Px = ntp["px"].array()
        self.Py = ntp["py"].array()
        self.Pz = ntp["pz"].array()
        self.E = ntp["E"].array()
        self.nTrk = ntp["nTrk"].array()
        file.close()
    def sign(self, x):
        return -1 if x<0 else 1
    def get_p_from_list(self, iEv, part_list):
        p = TLorentzVector(0, 0, 0, 0)
        for i in part_list:
            if( abs(max(part_list)) >= self.nTrk[iEv]):
                continue
            abs_i = abs(i)
            p += self.sign(i)*TLorentzVector(
                self.Px[iEv][abs_i], self.Py[iEv][abs_i], self.Pz[iEv][abs_i], self.E[iEv][abs_i]
            )
        return p
    def size(self):
        return len(self.Px)
    def apply(self, func, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        return p.func        
    def m2(self, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        return p.mag2
    def m(self, iEv, part_list):
        try:
            p = self.get_p_from_list(iEv, part_list)
            return p.mag
        except ValueError:
            return 0
    def pt2(self, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        return p.pt2
    def pt(self, iEv, part_list):
        try:
            p = self.get_p_from_list(iEv, part_list)
            return p.pt
        except ValueError:
            return 0
        

In [ ]:
df = rootDF('../build/evtOutput.root')

In [ ]:
part_list = [2, 3]
pt2_list = [df.pt2(iEv, part_list) for iEv in range( df.size() )]
pt2_list = np.array([pt2 for pt2 in pt2_list if pt2])

In [ ]:
part_list = [0, -1]
pt_list = [df.pt(iEv, part_list) for iEv in range(df.size() )]
pt_list = np.array([pt for pt in pt_list if pt])

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows = 1)
axes[0].hist(pt2_list);
axes[1].hist(pt_list**2);
plt.show()

In [ ]:
len(pt2_list)

In [ ]:
len(pt_list)

In [ ]:
pt2_list[8]

In [ ]:
pt_list[8]**2

In [ ]:
plt.hist(pt2_list, bins=30);

In [ ]:
plt.hist(pt_list, bins=30);

In [ ]:
plt.hist(np.sqrt(pt2_list), bins = 30);